In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from random import shuffle
import glob
from sklearn.preprocessing import LabelEncoder as le
#from classification_models.resnet import ResNet18
from sklearn.model_selection import train_test_split

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.applications import MobileNet,densenet
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import concatenate, add
from keras.layers import Input


In [ ]:
# cla = [x[1] for x in os.walk("../input/vision_asgn2/vision_asgn2")][0]

In [ ]:
data = []
y = []
fine = True
kk = glob.glob('../input/train/train/*/*/*.jpg')
shuffle(kk)
for i in kk:
    data.append(cv2.resize(mpimg.imread(i)/255.0 ,(224,224)))
    pp = i.split('/')
    if fine == True:
        y.append(pp[4] + '_' + pp[5])
    else:
        y.append(pp[4])
data = np.array(data)

# kk


In [ ]:
le_ = le()
le_.fit(y)
y = le_.transform(y)

In [ ]:
x_train,x_test,y_train_,y_test_ = train_test_split(data,y,test_size = .1)

In [ ]:
y_train_ = keras.utils.to_categorical(y_train_, 36)
y_test_ = keras.utils.to_categorical(y_test_, 36)

In [ ]:
x_train.shape

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=30,
    horizontal_flip=True,
    zoom_range=0.2)
datagen.fit(x_train)

In [ ]:
'Done'

In [ ]:
inp = Input(shape = (224,224,3))

In [ ]:
base_model1=MobileNet(weights='imagenet',include_top=False) 
x=base_model1(inp)
x = Conv2D(512, (3, 3), activation='relu')(x)
#x = MaxPooling2D(pool_size=(2, 2))
x1=GlobalAveragePooling2D()(x)
# x=Dense(512,activation='relu')(x) 
# preds=Dense(5,activation='softmax')(x) 
#model1 = Model(inputs=base_model1.input, outputs=x1)
#model1.summary()

In [ ]:
#base_model2 = ResNet18((224, 224, 3), weights='imagenet', include_top=False)
base_model2 = densenet.DenseNet121(include_top = False, weights='imagenet')

x = base_model2(inp)
x = Conv2D(512, (3, 3), activation='relu')(x)
x2=GlobalAveragePooling2D()(x)
#model2=Model(inputs=base_model2.input,outputs=x2)
# len(model.layers)
#model2.summary()

In [ ]:
concatenated = concatenate([x1, x2])
concat_out = Dense(1024, activation='relu')(concatenated)
#concat_out = Dense(1024,activation = 'relu')(concat_out)
concat_out_ = Dense(36, activation='softmax')(concat_out)
merged_model= Model(inputs=inp, outputs=concat_out_)

merged_model.summary()

In [ ]:
'Done'

In [ ]:
len(merged_model.layers)

In [ ]:
merged_model.layers[:3]

In [ ]:
model = merged_model

In [ ]:
# m= Merge([base_model1, base_model2,], mode='concat')

In [ ]:
# model=Model(inputs=base_model.input,outputs=preds)
# len(model.layers)

In [ ]:
for layer in model.layers[:3]:
    layer.trainable=False
for layer in model.layers[3:]:
    layer.trainable=True
print(model.summary())

In [ ]:
?Adam

In [ ]:
optimizer = Adam(lr = 0.0005,decay=0.1,amsgrad=True)
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#callbacks = [EarlyStopping(monitor='val_loss', patience=5)]

In [ ]:
model.fit_generator(datagen.flow(x_train,y_train_,batch_size = 64),
                    steps_per_epoch= len(x_train)/64,
                    epochs = 50,
                    #callbacks=callbacks,
                   validation_data = (x_test,y_test_))

In [ ]:
del data

In [ ]:
# epochs = 25
# for e in range(epochs):
#     batches = 0
#     for x_batch, y_batch in datagen.flow(x_train,y_train_,batch_size = 64):
#         model.fit([x_batch, x_batch], y_batch)
#         batches += 1
        
#         if(batches > len(x_train)/64):
#             break

In [ ]:
np.argmax(model.predict([x_test, x_test]),axis=1)

In [ ]:
np.argmax(y_test_,axis=1)

In [ ]:
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(256,256,3)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(128, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(36, activation='softmax'))

In [ ]:
model1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model1.fit(data, y_train,
          batch_size=68,
          epochs=5)


In [ ]:
model.summary()